In [1]:
import zipfile

zip_path = r'E:\Project\iris detection\archive (1).zip'  # <- raw string
extract_path = 'iris_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")


Extraction complete.


In [2]:
import os

full_path = os.path.abspath("E:\Project\iris detection")
print("Extracted folder path:", full_path)


Extracted folder path: E:\Project\iris detection


In [3]:
import os
import cv2
import numpy as np

data = []
labels = []

root_folder = r"E:\Project\iris detection\iris_dataset"

# Traverse all folders
for person_folder in os.listdir(root_folder):
    person_path = os.path.join(root_folder, person_folder)
    if os.path.isdir(person_path):
        for img_file in os.listdir(person_path):
            img_path = os.path.join(person_path, img_file)
            # Load image (grayscale or color based on your model)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # or cv2.IMREAD_COLOR
            if img is not None:
                img_resized = cv2.resize(img, (100, 100))  # Resize for consistency
                data.append(img_resized)
                labels.append(person_folder)  # use folder name as label


In [4]:
from sklearn.preprocessing import LabelEncoder

X = np.array(data).reshape(len(data), 100, 100, 1)  # If grayscale; use (100,100,3) if color
le = LabelEncoder()
y = le.fit_transform(labels)


In [5]:
import os

root_folder =  r"E:\Project\iris detection\iris_dataset"
print("Total folders (people):", len(os.listdir(root_folder)))

# List a few folders for confirmation
print("Sample folders:", os.listdir(root_folder)[:5])


Total folders (people): 1000
Sample folders: ['000', '001', '002', '003', '004']


In [6]:
import os
import cv2
import numpy as np

data = []
labels = []

root_folder = r"E:\Project\iris detection\iris_dataset" # Update if needed

for person_folder in os.listdir(root_folder):
    person_path = os.path.join(root_folder, person_folder)
    if os.path.isdir(person_path):
        for img_file in os.listdir(person_path):
            img_path = os.path.join(person_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (100, 100))
                data.append(img)
                labels.append(person_folder)

print(f"✅ Loaded {len(data)} images.")
X = np.array(data).reshape(-1, 100, 100, 1)
y = np.array(labels)


✅ Loaded 10000 images.


In [7]:
print("Total images loaded:", len(data))


Total images loaded: 10000


In [8]:
data = np.array(data)
labels = np.array(labels)


In [9]:
X = np.array(data).reshape(-1, 100, 100, 1)
y = np.array(labels)


In [10]:
from sklearn.preprocessing import LabelEncoder

X = np.array(data).reshape(len(data), 100, 100, 1)  # If grayscale; use (100,100,3) if color
le = LabelEncoder()
y = le.fit_transform(labels)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
import os
import random
import cv2
import numpy as np
from itertools import combinations

def load_image(path, size=(100, 100)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        return cv2.resize(img, size)
    return None

def generate_image_pairs(root_dir, image_size=(100, 100), num_pairs_per_person=5):
    pairs = []
    labels = []

    people = [p for p in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, p))]
    people.sort()

    # Generate SAME pairs
    for person in people:
        person_path = os.path.join(root_dir, person)
        images = [os.path.join(person_path, img) for img in os.listdir(person_path)
                  if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

        combinations_same = list(combinations(images, 2))
        random.shuffle(combinations_same)

        for pair in combinations_same[:num_pairs_per_person]:
            img1 = load_image(pair[0], image_size)
            img2 = load_image(pair[1], image_size)
            if img1 is not None and img2 is not None:
                pairs.append([img1, img2])
                labels.append(1)  # Same person

    # Generate DIFFERENT pairs
    for _ in range(len(pairs)):
        person1, person2 = random.sample(people, 2)
        img1_path = random.choice(os.listdir(os.path.join(root_dir, person1)))
        img2_path = random.choice(os.listdir(os.path.join(root_dir, person2)))

        img1 = load_image(os.path.join(root_dir, person1, img1_path), image_size)
        img2 = load_image(os.path.join(root_dir, person2, img2_path), image_size)
        if img1 is not None and img2 is not None:
            pairs.append([img1, img2])
            labels.append(0)  # Different person

    pairs = np.array(pairs)
    labels = np.array(labels)
    return [pairs[:, 0], pairs[:, 1]], labels


In [13]:
import tensorflow as tf
import numpy as np

print("✅ TensorFlow version:", tf.__version__)
print("✅ NumPy version:", np.__version__)


✅ TensorFlow version: 2.10.0
✅ NumPy version: 1.23.5


In [14]:
import os
import cv2
import numpy as np
import random
from itertools import combinations
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input, Model
import tensorflow.keras.backend as K


In [15]:
def build_siamese_model(input_shape=(100, 100, 1)):
    def build_base_cnn():
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            layers.MaxPooling2D(),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(128, activation='relu')
        ])
        return model

    base_cnn = build_base_cnn()

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    feature_a = base_cnn(input_a)
    feature_b = base_cnn(input_b)

    # L1 distance
    distance = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([feature_a, feature_b])
    output = layers.Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[input_a, input_b], outputs=output)
    return model


In [16]:
# Set dataset path
root_dir = r"E:\Project\iris detection\iris_dataset"  # Change as per your path
(X1, X2), y = generate_image_pairs(root_dir)

# Preprocess
X1 = np.expand_dims(np.array(X1) / 255.0, axis=-1)
X2 = np.expand_dims(np.array(X2) / 255.0, axis=-1)
y = np.array(y)

# Split
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    X1, X2, y, test_size=0.2, random_state=42
)

# Train model
model = build_siamese_model(input_shape=(100, 100, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit([X1_train, X2_train], y_train, validation_data=([X1_test, X2_test], y_test), epochs=10, batch_size=32)


Epoch 1/10
250/250 [==============================] - 80s 316ms/step - loss: 0.1535 - accuracy: 0.9262 - val_loss: 0.0787 - val_accuracy: 0.9785
Epoch 2/10
250/250 [==============================] - 78s 314ms/step - loss: 0.0469 - accuracy: 0.9887 - val_loss: 0.0416 - val_accuracy: 0.9930
Epoch 3/10
250/250 [==============================] - 112s 449ms/step - loss: 0.0372 - accuracy: 0.9933 - val_loss: 0.0560 - val_accuracy: 0.9930
Epoch 4/10
250/250 [==============================] - 196s 786ms/step - loss: 0.0249 - accuracy: 0.9954 - val_loss: 0.0390 - val_accuracy: 0.9935
Epoch 5/10
250/250 [==============================] - 80s 322ms/step - loss: 0.0147 - accuracy: 0.9974 - val_loss: 0.0449 - val_accuracy: 0.9920
Epoch 6/10
250/250 [==============================] - 81s 322ms/step - loss: 0.0164 - accuracy: 0.9979 - val_loss: 0.0483 - val_accuracy: 0.9845
Epoch 7/10
250/250 [==============================] - 80s 319ms/step - loss: 0.0129 - accuracy: 0.9977 - val_loss: 0.0308 - val_

In [17]:
# ✅ Save the model
model.save("siamese_model.h5")
print("✅ Model saved as siamese_model.h5")


✅ Model saved as siamese_model.h5


In [18]:
model.save("siamese_model.h5")


In [19]:
def predict_same_person(img1_path, img2_path, model, image_size=(100, 100)):
    def preprocess(path):
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        img = img.astype('float32') / 255.0
        return np.expand_dims(img, axis=(0, -1))

    img1 = preprocess(img1_path)
    img2 = preprocess(img2_path)

    prediction = model.predict([img1, img2])[0][0]
    if prediction > 0.5:
        return f"✅ Same person (Confidence: {prediction:.2f})"
    else:
        return f"❌ Different persons (Confidence: {1 - prediction:.2f})"


In [23]:
def predict_same_person(model, image_size=(100, 100)):
    img1_path = r"E:\Project\iris detection\iris_dataset\001\S6001S00.jpg"
    img2_path = r"E:\Project\iris detection\iris_dataset\001\S6001S03.jpg"

    def preprocess(path):
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        img = img.astype('float32') / 255.0
        return np.expand_dims(img, axis=(0, -1))

    img1 = preprocess(img1_path)
    img2 = preprocess(img2_path)

    prediction = model.predict([img1, img2])[0][0]
    if prediction > 0.5:
        return f"✅ Same person (Confidence: {prediction:.2f})"
    else:
        return f"❌ Different persons (Confidence: {1 - prediction:.2f})"
    



In [24]:
print(predict_same_person(model))

1/1 [==============================] - 0s 149ms/step
✅ Same person (Confidence: 1.00)
